In [9]:
import json
import pickle
import numpy as np

In [ ]:
import os, sys
sys.path.append("../volta")
from volta.datasets.flickr30ke_ablation_dataset import get_annotations, get_sentence_data

In [17]:
with open("/science/image/nlp-datasets/emanuele/data/flickr30k_entities/val.txt", "r") as f:
    images = f.read().splitlines()

entries = []
for img in images:
    annotation = get_annotations(os.path.join("/science/image/nlp-datasets/emanuele/data/flickr30k_entities/Annotations", img + ".xml"))
    sentences = get_sentence_data(os.path.join("/science/image/nlp-datasets/emanuele/data/flickr30k_entities/Sentences", img + ".txt"))
    for i, sent in enumerate(sentences):
        for phrase in sent["phrases"]:
            if str(phrase["phrase_id"]) in annotation["boxes"].keys():
                entries.append(
                    {
                        "caption": phrase["phrase"],
                        "first_word_index": phrase["first_word_index"],
                        "sent_id": int(phrase["phrase_id"]),
                        "image_id": int(img),
                        "refBoxes": annotation["boxes"][str(phrase["phrase_id"])],
                        "sentence": sent["sentence"],
                        "type": phrase["phrase_type"],
                    }
                )

In [25]:
# unique types
types_set = {t for e in entries for t in e['type']}
types_set

{'animals',
 'bodyparts',
 'clothing',
 'instruments',
 'other',
 'people',
 'scene',
 'vehicles'}

In [28]:
type2cnt = {t: 0 for t in types_set}
for entry in entries:
    for t in entry['type']:
        type2cnt[t] += 1
type2cnt

{'vehicles': 338,
 'scene': 1533,
 'instruments': 155,
 'people': 5813,
 'animals': 523,
 'other': 3284,
 'bodyparts': 541,
 'clothing': 2335}

In [21]:
mul_type_entris = []
for entry in entries:
    if len(entry['type']) != 1:
        mul_type_entris.append(entry)
len(mul_type_entris), len(entries)

(89, 14433)

In [24]:
# at most 2 types
for e in mul_type_entris:
    if len(e['type']) != 2:
        print(e)
        break

In [30]:
# 70/89 have "scene" as 2nd type, while the others have "people"
ppl_cnt = 0
for e in mul_type_entris:
    if 'scene' not in e['type']:
        print(e['type'])
        ppl_cnt += 1
print(ppl_cnt)

['bodyparts', 'people']
['animals', 'people']
['animals', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['animals', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['animals', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
['bodyparts', 'people']
19


In [32]:
# we just select the first type (more specific) when we have 2
ix2type = {ix: e['type'][0] for ix, e in enumerate(entries)}
len(ix2type)

14433

In [33]:
ix2type[0]

'people'

In [34]:
with open("ix2type.pkl", "wb") as f:
    pickle.dump(ix2type, f)